<a href="https://colab.research.google.com/github/syno3/Data-science-portfolio/blob/dev/smsSpamClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### importing the required modules

import tensorflow as tf
import pandas as pd
from tensorflow import keras

import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import utils
import csv
import pandas as pd

In [ ]:
# get data files
#loading the data

!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
## converting to dataFrame and Preprocessing

train_df = pd.read_csv(train_file_path, sep='\t', names=['Classification', 'Text'])
test_df = pd.read_csv(test_file_path, sep='\t', names=['Classification', 'Text'])

In [21]:
## Mapping the data

x_train = train_df.drop('Text', axis=1).astype('category')
y_train = train_df.drop('Classification', axis=1)

x_test = test_df.drop('Text', axis=1).astype('category')
y_test = test_df.drop('Classification', axis=1)

In [22]:
### encoding the spam and ham into [0,1]

x_train =  x_train['Classification'].cat.codes
x_test = x_test['Classification'].cat.codes

In [25]:
## iterate over column (text) and encode raw text
VOCAB_SIZE = 1000
for i in y_train:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(y_train[i]))

In [29]:
for j in y_test:
  encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
      max_tokens=VOCAB_SIZE)
  encoder.adapt(np.array(y_test[j]))

In [31]:
vocab = np.array(encoder.get_vocabulary())

In [45]:
### encode the text into indexes (y_train)
for i in y_train:
  data_train = y_train[i]
  encoded_y_train = encoder(data_train)[:3].numpy()


In [46]:
### encode the text into indexes (y_train)
for i in y_test:
  data_test = y_test[i]
  encoded_y_test = encoder(data_test)[:3].numpy()

In [47]:
for n in range(3):
  print("Original: ", data_train[n])
  print("Round-trip: ", " ".join(vocab[encoded_y_train[n]]))
  print()

Original:  ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.
Round-trip:  [UNK] [UNK] [UNK] a bad [UNK] about u [UNK] i dont like u right now i didnt know anything about [UNK] night but i guess im up for it                                                                                                                                              

Original:  you can never do nothing
Round-trip:  you can never do nothing                                                                                                                                                                      

Original:  now u sound like manky scouse boy steve,like! i is travelling on da bus home.wot has u inmind 4 recreation dis eve?
Round-trip:  now u [UNK] like [UNK] [UNK] boy [UNK] i is [UNK] on da bus [UNK] has u [UNK] 4 [UNK] dis [UNK]                                                                             

In [48]:
#### creat the model

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [71]:
# predict on a sample text without padding.

sample_text = ("how are you doing today?")
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.00501844]


In [72]:
# predict on a sample text with padding
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.00501844]


In [60]:
###compiling the model

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])